# Invoke functions from module

In [ ]:
import sys
sys.path.append('./modules')

In [ ]:
from script_module_02a import save_property, filter_properties

In [ ]:
filter_properties()

# Pass MLRun context and run function from module

In [ ]:
from mlrun import get_or_create_ctx
mlrun_ctx = get_or_create_ctx('mlrun_test')

In [ ]:
source_file = "/User/igztraining/mlrun/data/demo.csv"
target_file = "/User/igztraining/mlrun/data/condos.csv" 

In [ ]:
from module_handler_functions import save_property

In [ ]:
save_property(mlrun_ctx,source_file=source_file,target_file=target_file)

# Run local with MLRun

In [ ]:
from mlrun import run_local

In [ ]:
params = {"source_file" : "/User/igztraining/mlrun/data/demo.csv",
"target_file" : "/User/igztraining/mlrun/data/condos.csv" }

In [ ]:
run_result = run_local(name='filter_properties',project='Iguazio-Training',params=params,handler='filter_properties',command='modules/module_handler_functions.py')

## Run local with MLRun sdk hooks

In [ ]:
run_result = run_local(name='save_property',artifact_path='/User/artifacts',project='Iguazio-Training',params=params,handler='save_property',command='modules/mlrun_functions.ipynb')

# Deploy code as an MLRun function

In [ ]:
from mlrun import code_to_function, mlconf,mount_v3io

In [ ]:
fn = code_to_function(project='Iguazio-Training',name='save_properities', kind='job',filename='modules/mlrun_functions.ipynb')

In [ ]:
fn.spec.image = "mlrun/mlrun"

In [ ]:
fn.apply(mount_v3io())

In [ ]:
print(fn.to_yaml())

### Run interactive

In [ ]:
sp_run = fn.run(name='run_withcode',artifact_path="/User/artifacts",handler='save_property',params=params,project='Iguazio-Training')

In [ ]:
sp_run.outputs

### Run and forget

In [ ]:
sp_run = fn.run(name='run_withcode',artifact_path="/User/artifacts",handler='save_property',params=params,project='Iguazio-Training',watch=False)

# Run function as part of a pipeline

In [ ]:
import kfp
from kfp import dsl
from mlrun import run_pipeline

In [ ]:
@dsl.pipeline(
    name = 'Property Management',
    description = 'Condo reviews'
)
def job_pipeline(
source_file :str = ''
) -> None:
    """Define our pipeline.
    :param p1: A model parameter.
    """
    save_properties = fn.as_step(handler='save_property',
                            params={"source_file" : source_file,
"target_file" : "/User/igztraining/mlrun/data/condos.csv" },outputs=['condo_properties'])
    filter_properties = fn.as_step(handler='filter_properties',
                                   params={"source_file" : source_file,
"target_file" : "/User/igztraining/mlrun/data/condos.csv" },
                            inputs={"condo_properties":save_properties.outputs["condo_properties"]})

In [ ]:
kfp.compiler.Compiler().compile(job_pipeline, 'jobpipe.yaml')
artifact_path = 'v3io:///users/admin/kfp/{{workflow.uid}}/'
arguments = {"source_file" : "/User/igztraining/mlrun/data/demo.csv"}
run_id = run_pipeline(job_pipeline, arguments, experiment='property_management', artifact_path=artifact_path)